In [1]:
import numpy as np
import pandas as pd

In [2]:
class Blocking_System:
    def __init__(self, m: int, service, arrival, 
                 arrival_method: str = 'poisson', service_method: str = 'exp'): 
        self.clock=0.0                      #simulation clock
        self.num_service_units=m            #system with m service units
        self.arrival_mtd=arrival_method     #arrival distribution
        self.service_mtd=service_method     #service distribution
        self.param_service=service          #parameter for service dist.
        self.param_arrival=arrival          #parameter for arrival dist.
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_arrival_time()   #time of next arrival
        self.t_departures=np.ones(m)*100000. #departure times for each service unit (100.000 as infinite)
        self.dep_sums=np.zeros((m,), dtype=int) #Sum of service time
        self.states=np.zeros((m,), dtype=int) #current states
        self.num_of_departures=np.zeros((m,), dtype=int) #number of customers served
        self.lost_customers=0               #customers who left without service
        self.num_in_system=0                #customers in the system


    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival,min(self.t_departures))  
        self.clock=t_next_event

        t_departure=min(self.t_departures)
        idx = list(self.t_departures).index(t_departure)
        if self.t_arrival<t_departure:
            self.arrival()
        else:
            self.departure(idx)


    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        accepted = False
        for idx, status in enumerate(self.states):
            if status==0:
                accepted = True
                self.dep=self.gen_service_time()
                self.dep_sums[idx] += self.dep
                self.t_departures[idx]=self.clock + self.dep
                break

        self.t_arrival=self.clock+self.gen_arrival_time()
        if not accepted:
            self.lost_customers += 1


    def departure(self, idx: int):
        self.num_of_departures[idx] += 1
        self.t_departures[idx]=100000. # (100.000 as infinite)
        self.states[idx]=0                  


    def gen_arrival_time(self):         #function to generate arrival times 
        if self.arrival_mtd=='erlang':
            return (np.random.gamma(self.param_arrival)) # Erlang distribution (using Gamma with shape=int)
        elif self.arrival_mtd=='hyperexp': # Hyper Exponential distribution p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0
            if np.random.uniform() <= 0.8: #p1
                return (np.random.exponential(scale=1./0.833)) #λ1
            else: #p2
                return (np.random.exponential(scale=1./5.)) #λ2
        else:
            return (np.random.poisson()) # Poisson distribution

    
    def gen_service_time(self):         #function to generate service time
        if self.service_mtd=='constant':
            return self.param_service
        if self.service_mtd=='pareto': # Pareto distribution
            return (np.random.pareto(self.param_service))
        if self.service_mtd=='normal':
            return (np.random.normal(loc=self.param_service)) # Normal Distribution
        else:
            return (np.random.exponential()) # Exponential distribution (lamda=1)
    

Write a discrete event simulation program for a blocking system, i.e. a system with m service units and no waiting room. The offered traffic A is the product of the mean arrival rate and the mean service time.

1. The arrival process is modelled as a Poisson process. Report the fraction of blocked customers, and a confidence interval for this fraction. Choose the service time distribution as exponential. Parameters: m = 10, mean service time = 8 time units, mean time between customers = 1 time unit (corresponding to an offered traffic of 8 erlang), 10 x 10.000 customers.

In [3]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival)
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival)
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p1.xlsx')   

<ipython-input-3-9f26462e99ff>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-3-9f26462e99ff>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-3-9f26462e99ff>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-3-9f26462e99ff>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


2. The arrival process is modelled as a renewal process using the same parameters as in Part 1 when possible. Report the fraction of blocked customers, and a confidence interval for this fraction for at least the following two cases
- (a) Experiment with Erlang distributed inter arrival times The Erlang distribution should have a mean of 1

In [4]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, arrival_method='erlang')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, arrival_method='erlang')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_arrivals,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p2a.xlsx')   

<ipython-input-4-74bd704baf1c>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-4-74bd704baf1c>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-4-74bd704baf1c>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-4-74bd704baf1c>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


2. The arrival process is modelled as a renewal process using the same parameters as in Part 1 when possible. Report the fraction of blocked customers, and a confidence interval for this fraction for at least the following two cases
- (b) hyper exponential inter arrival times. The parameters for the hyper exponential distribution should be p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0.

In [5]:
m = 10
mu_service = 8
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, arrival_method='hyperexp')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, arrival_method='hyperexp')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_in_system,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p2b.xlsx')   

<ipython-input-5-7b164fbdf879>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-5-7b164fbdf879>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-5-7b164fbdf879>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-5-7b164fbdf879>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


3. The arrival process is again a Poisson process like in Part 1. Experiment with different service time distributions with the same mean service time and m as in Part 1 and Part 2.
- (a) Constant service time
- (b) Pareto distributed service times with at least k = 1.05 and k = 2.05.
- (c) Choose one or two other distributions.

In [6]:
m = 10
mu_service = 8.
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, service_method='constant')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, service_method='constant')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_in_system,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p3a.xlsx')   

<ipython-input-6-964008fb1b3b>:20: RuntimeWarning: divide by zero encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-6-964008fb1b3b>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-6-964008fb1b3b>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-6-964008fb1b3b>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-6-964008fb1b3b>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


In [7]:
m = 10
k = 1.05
mu_arrival = 1
s=Blocking_System(m, k, mu_arrival, service_method='pareto')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, k, mu_arrival, service_method='pareto')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_in_system,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p3b1.xlsx')   

<ipython-input-7-924c2277d332>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-7-924c2277d332>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-7-924c2277d332>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-7-924c2277d332>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


In [8]:
m = 10
k = 2.05
mu_arrival = 1
s=Blocking_System(m, k, mu_arrival, service_method='pareto')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, k, mu_arrival, service_method='pareto')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_in_system,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p3b2.xlsx')   

<ipython-input-8-deebd5346fa0>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-8-deebd5346fa0>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-8-deebd5346fa0>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-8-deebd5346fa0>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


In [9]:
m = 10
mu_service = 8.
mu_arrival = 1
s=Blocking_System(m, mu_service, mu_arrival, service_method='normal')
df=pd.DataFrame(columns=['Fraction of blocked customers','Average interarrival time','Total Customers','Blocked Customers',
                         'Average service time unit 1','Average service time unit 2',
                         'Average service time unit 3','Average service time unit 4','Average service time unit 5',
                         'Average service time unit 6','Average service time unit 7','Average service time unit 8',
                         'Average service time unit 9','Average service time unit 10',
                         'Utilization unit 1','Utilization unit 2','Utilization unit 3','Utilization unit 4',
                         'Utilization unit 5','Utilization unit 6','Utilization unit 7','Utilization unit 8',
                         'Utilization unit 9','Utilization unit 10'])

for i in range(10):
    np.random.seed(i)
    s.__init__(m, mu_service, mu_arrival, service_method='normal')
    while s.clock <= 10000 :
        s.time_adv() 
    a=pd.Series([s.lost_customers/s.num_in_system,s.clock/s.num_arrivals,s.num_arrivals,s.lost_customers,
                 s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
                 s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
                 s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
                 s.dep_sums[9]/s.num_of_departures[9],
                 s.dep_sums[0]/s.clock,s.dep_sums[1]/s.clock,s.dep_sums[2]/s.clock,s.dep_sums[3]/s.clock,s.dep_sums[4]/s.clock,
                 s.dep_sums[5]/s.clock,s.dep_sums[6]/s.clock,s.dep_sums[7]/s.clock,s.dep_sums[8]/s.clock,s.dep_sums[9]/s.clock
                 ],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('ex4_p3c.xlsx')   

<ipython-input-9-2a6ad09c5a33>:20: RuntimeWarning: divide by zero encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-9-2a6ad09c5a33>:20: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[0]/s.num_of_departures[0],s.dep_sums[1]/s.num_of_departures[1],s.dep_sums[2]/s.num_of_departures[2],
<ipython-input-9-2a6ad09c5a33>:21: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[3]/s.num_of_departures[3],s.dep_sums[4]/s.num_of_departures[4],s.dep_sums[5]/s.num_of_departures[5],
<ipython-input-9-2a6ad09c5a33>:22: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[6]/s.num_of_departures[6],s.dep_sums[7]/s.num_of_departures[7],s.dep_sums[8]/s.num_of_departures[8],
<ipython-input-9-2a6ad09c5a33>:23: RuntimeWarning: invalid value encountered in long_scalars
  s.dep_sums[9]/s.num_of_departures[9],


4. Compare confidence intervals for Parts 1, 2, and 3 and try explain differences if any.

**Response**
